In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets, models
import torchvision.transforms as TF
from collections import Counter
from tqdm import tqdm 
# print(torch.cuda.is_available(), torch.cuda.current_device())
import sys
sys.path.append('../src/')

from inaturalist_dataset import WrapNaturalist
    
class inaturalist_normalize():
    def __init__(self):
        self.tf = TF.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

    def __call__(self, img):
        return self.tf(img)

    
class inaturalist_train():
    """Pre-processing for inaturalist training.
    """
    # TODO maybe the tranform topilimage is messing up the normalization!
    def __init__(self):
        self.tf = TF.Compose([TF.Resize(256), TF.CenterCrop(224), TF.RandomHorizontalFlip(), TF.ToTensor(), 
                              inaturalist_normalize(),
                              ])
        # self.tf = TF.Compose([core50_normalize()])

    def __call__(self, img):
        return self.tf(img)
    
    
    
def exclude_problem_indices_inaturalist_2019(dataroot,transform=None):
    
    dataset = datasets.INaturalist(dataroot, version='2019', target_type=['super'], \
                                transform=transform, target_transform=None, download=False)
    
    indices_all = np.arange(dataset.__len__())
    exclude_inds = []
    
    for i, ind in enumerate(tqdm(indices_all)):
        try:
            _ = dataset.__getitem__(ind)
        except RuntimeError:
            exclude_inds.append(i)
    
    if len(exclude_inds)>0:
        print('issue inds', exclude_inds, len(exclude_inds))
        keep = np.setdiff1d(np.arange(indices_all.shape[0]), np.array(exclude_inds))
        indices_all = indices_all[keep]
        
    np.save(dataroot + '/'+'2019_valid_indices.npy', indices_all)
    
    return indices_all
    
tapply = inaturalist_train()

dataroot = '/lab/arios/data/inaturalist/'


# indices_all = exclude_problem_indices_inaturalist_2019(dataroot, transform=tapply)
# dset = datasets.INaturalist(dataroot, version='2019', target_type=['super'], \
#     transform=tapply, target_transform=None, download=False)


dset= WrapNaturalist(dataroot, version='2021_train_mini', target_type=['phylum'], \
    transform=tapply, target_transform=None, download=False)


# inds = np.load(dataroot+'/'+'2019_valid_indices.npy')


print(dset.__len__())

500000


In [8]:
dataroot = '/lab/arios/data/inaturalist/'
class inaturalist_normalize():
    def __init__(self):
        self.tf = TF.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

    def __call__(self, img):
        return self.tf(img)


class inaturalist_train():
    """Pre-processing for inaturalist training.
    """
    # TODO maybe the tranform topilimage is messing up the normalization!
    def __init__(self):
        self.tf = TF.Compose([TF.Resize(256), TF.CenterCrop(224), TF.RandomHorizontalFlip(), TF.ToTensor(), 
                            #   inaturalist_normalize(),
                              ])
        # self.tf = TF.Compose([core50_normalize()])

    def __call__(self, img):
        return self.tf(img)
    
tapply = inaturalist_train()



# dset = datasets.INaturalist(dataroot, version='2019', target_type=['super'], \
#     transform=tapply, target_transform=None, download=False)

print(dset.__len__())

AttributeError: 

In [121]:
from typing import Any, Callable, Dict, List, Optional, Union, Tuple

class WrapIndex(datasets.INaturalist):
    
    def __init__(self, root: str, version: str = "2021_train", target_type: Union[List[str], str] = "full", transform: Optional[Callable] = None, target_transform: Optional[Callable] = None, download: bool = False) -> None:
        super().__init__(root, version, target_type, transform, target_transform, download)
        
    def __getitem__(self, index):

        cat_id, _ = self.index[index]

        target = []
        target_name = []
        for t in self.target_type:
            if t == "full":
                target.append(cat_id)
            else:
                target.append(self.categories_map[cat_id][t])
                if '21' in self.version:
                    target_name.append(self.all_categories[cat_id].split('_')[2])
        target = tuple(target) if len(target) > 1 else target[0]

        if self.target_transform is not None:
            target = self.target_transform(target)


        if '21' in self.version:
            return target, target_name
        
        return target
    
    
    def __getindices_2019__(self):
        
        self.labels = np.zeros((self.__len__(),)).astype(int)
        for i in range(self.__len__()):
            target = self.__getitem__(i)            
            self.labels[i]=target
            
        return self.labels
            
            
    def __getindices_2021__(self):
        self.labels_map =[]
        self.labels = np.zeros((self.__len__(),)).astype(int)
        for i in range(self.__len__()):
            target, name = self.__getitem__(i)            
            self.labels[i]=target
            
            self.labels_map.append((target, name))
            
        self.labels_map = list(set(self.labels_map))
        
        
        return self.labels
    
    
dset_index = WrapIndex(dataroot, version='2021_train_mini', target_type=['phylum'], \
    transform=tapply, target_transform=None, download=False)


# dset_index = WrapIndex(dataroot, version='2019', target_type=['super'], \
#     transform=tapply, target_transform=None, download=False)

In [122]:
labels = dset_index.__getindices_2019__()

In [125]:
Counter(labels)

Counter({2: 2284, 3: 41204, 0: 3912, 5: 14513, 1: 47867, 4: 158463})

In [114]:
loader = torch.utils.data.DataLoader(dset_index, batch_size=100,shuffle=False, num_workers=4)
y = np.zeros((dset_index.__len__(),)).astype(int)
y_names = []
y_match=[]
for i, batch in enumerate(loader):
    
    # y_vals, y_n = batch
    
    y_vals= batch

            
    y[i*100:i*100+100]=y_vals.numpy()
    
    # y_names.extend(list(y_n[0]))
    
    
    # y_match.extend(list(zip(y_vals.numpy().tolist(), list(y_n[0]))))
    
    
    
# list(set(y_match))
    # break
    
# y_names

np.unique(y)


array([0, 1, 2, 3, 4, 5])

In [98]:

cat_name,_ = dset.index[100]
dset_index.categories_map[cat_name]['phylum']

0

In [104]:
list(set(y_match))

[(6, 'Ascomycota'),
 (12, 'Tracheophyta'),
 (9, 'Chlorophyta'),
 (3, 'Cnidaria'),
 (11, 'Rhodophyta'),
 (2, 'Chordata'),
 (7, 'Basidiomycota'),
 (4, 'Echinodermata'),
 (1, 'Arthropoda'),
 (0, 'Annelida'),
 (10, 'Marchantiophyta'),
 (8, 'Bryophyta'),
 (5, 'Mollusca')]

In [102]:
from collections import Counter

Counter(y.tolist())

Counter({0: 200,
         1: 137600,
         2: 120800,
         3: 1300,
         4: 1050,
         5: 8450,
         6: 4200,
         7: 12850,
         8: 1750,
         9: 200,
         10: 350,
         11: 350,
         12: 210900})

In [36]:

class inaturalist_normalize():
    def __init__(self):
        self.tf = TF.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

    def __call__(self, img):
        return self.tf(img)


class inaturalist_train():
    """Pre-processing for inaturalist training.
    """
    # TODO maybe the tranform topilimage is messing up the normalization!
    def __init__(self):
        self.tf = TF.Compose([TF.Resize(256), TF.CenterCrop(224), TF.RandomHorizontalFlip(), TF.ToTensor(), inaturalist_normalize()])
        # self.tf = TF.Compose([core50_normalize()])

    def __call__(self, img):
        return self.tf(img)
    
    
class inaturalist_test():
    """Pre-processing for inaturalist training.
    """
    # TODO maybe the tranform topilimage is messing up the normalization!
    def __init__(self):
        self.tf = TF.Compose([TF.Resize(256), TF.CenterCrop(224), TF.ToTensor(), inaturalist_normalize()])
        # self.tf = TF.Compose([core50_normalize()])

    def __call__(self, img):
        return self.tf(img)
    
    


In [37]:

dataroot = '/lab/arios/data/inaturalist/'

tapply = inaturalist_train()

dset = datasets.INaturalist(dataroot, version= '2019', target_type= 'super', download=False, transform=tapply)



In [22]:
loader = torch.utils.data.DataLoader(dset, batch_size=10, shuffle=True, num_workers=4)


batch = next(iter(loader))

In [23]:
batch[1]

tensor([1, 4, 4, 4, 3, 4, 4, 4, 5, 4])

In [33]:

cats = {0:0,1:0,2:0,3:0,4:0,5:0}

for pair in dset.index:
    cats[dset.categories_map[pair[0]]['super']]+=1


In [35]:
2284*0.8, 2284*0.2

(1827.2, 456.8)

In [5]:
# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

TypeError: 'str' object is not callable

In [39]:
dset.__len__()

268243